In [18]:
import pandas as pd
import numpy as np

In [19]:
#importing the terms csv data obtained from the internet and creating Dataframe
Germany_interest= pd.DataFrame(pd.read_csv("Germany_interest.csv", header =1))
UK_interest= pd.DataFrame(pd.read_csv("UK_interest.csv", header =1))
USA_interest= pd.DataFrame(pd.read_csv("USA_interest.csv", header =1))

#Note, I had to learn what header was in this

In [21]:
#Cleaning up term datasets 
#drop NA from AI term 

Germany_interest.dropna(inplace=True)

#drop NA from Chat gpt terms
UK_interest.dropna(inplace=True)

#drop NA from USA interest
USA_interest.dropna(inplace=True)

USA_interest.head(5)

,Week,ai: (United States)
0,2020-05-03,5
1,2020-05-10,5
2,2020-05-17,5
3,2020-05-24,5
4,2020-05-31,4


In [22]:
#renaming columns
Germany_interest.rename(columns={'ai: (Germany)': 'Popularity_of_search'} )
UK_interest.rename(columns={'ai: (United Kingdom)': 'Popularity_of_search'} )
USA_interest.rename(columns={'ai: (United States)': 'Popularity_of_search'} )

,Week,Popularity_of_search
0,2020-05-03,5
1,2020-05-10,5
2,2020-05-17,5
3,2020-05-24,5
4,2020-05-31,4
...,...,...
257,2025-04-06,96
258,2025-04-13,88
259,2025-04-20,94
260,2025-04-27,99
